In [1]:
!git clone https://github.com/nsavinov/semantic3dnet
%cd /content/semantic3dnet/build
!./setup.sh
# !./build.sh
!./build_run.sh

Cloning into 'semantic3dnet'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70
Unpacking objects: 100% (70/70), done.
/content/semantic3dnet/build
--2022-08-02 08:06:03--  http://www.semantic3d.net/data/point-clouds/training1/bildstein_station1_xyz_intensity_rgb.7z
Resolving www.semantic3d.net (www.semantic3d.net)... 129.132.89.156
Connecting to www.semantic3d.net (www.semantic3d.net)|129.132.89.156|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219277083 (209M) [application/x-7z-compressed]
Saving to: ‘bildstein_station1_xyz_intensity_rgb.7z’

bildstein_station1_ 100%[===================>] 209.12M  18.6MB/s    in 12s     

2022-08-02 08:06:15 (17.4 MB/s) - ‘bildstein_station1_xyz_intensity_rgb.7z’ saved [219277083/219277083]

--2022-08-02 08:06:15--  http://www.semantic3d.net/data/point-clouds/testing2/MarketplaceFeldkirch_Station4_rgb_intensity-reduced.txt.7z
Resolving www.semantic3d.net (www.semantic3d.

In [2]:
%cd /content/semantic3dnet/

/content/semantic3dnet


In [3]:
import torch
import torch.nn.functional as F
import numpy as np
import math
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline

In [4]:
cpp_options = {
    'kWindowSize': 16,
    'kBatchSize': 100,
    'kNumberOfClasses': 8,
    'kDefaultNumberOfScales': 5,
    'kDefaultNumberOfRotations': 1,
    'kSpatialResolution': 0.025,
    'kBatchResamplingLimit': 100,
    'kPointCloudVerbose': False,
    'kDefaultSeed': 1,
}

In [5]:
opt = {
    'experiment_name': 'small_train',
    'max_epochs': 20,
    'device': 'cuda:0',
    'resume_ckpt': None,
    'print_every_n': 100,
    'validate_every_n': 100,
    'kSide': cpp_options['kWindowSize'],
    'batch_size': cpp_options['kBatchSize'],
    'n_outputs': cpp_options['kNumberOfClasses'],
    'kNumberOfScales': cpp_options['kDefaultNumberOfScales'],
    'kNumberOfRotations': cpp_options['kDefaultNumberOfRotations'],
    'kSmallPrintingInterval': 2,
    'number_of_filters': 16,
    'kLargePrintingInterval': 100,
    'kWarmStart': False,
    'kModelDumpName': '../dump/model_dump',
    'kOptimStateDumpName': '../dump/optim_state_dump',
    'kStreamingPath': '../data/benchmark/sg28_station4_intensity_rgb_train.txt'
}

In [6]:
class PointDataset(torch.utils.data.Dataset):
    def __init__(self, file_name, count, opt):
        loaded_chunk = []
        loaded_tensor = None
        i = 0
        file1 = open(file_name, 'r')
        while True:
            i = i + 1
            chunks = []
            line = file1.readline()
            if not line:
                break
                
            for w in line.split():
                chunks.append(int(w))
            
            loaded_chunk.append(chunks)
            
            if i % 100 == 0 or i >= count:
                if loaded_tensor != None:
                    temp = torch.cat([loaded_tensor, torch.tensor(loaded_chunk)], 0)
                    loaded_tensor = temp
                else:
                    loaded_tensor = torch.tensor(loaded_chunk)
                loaded_chunk = []
                if i >= count:
                    break
        
        file1.close()
        
        loaded_data = loaded_tensor
        print(loaded_tensor.shape)
        count = loaded_data.size()[0]
        data_data = torch.clone(loaded_data[:, :-1]).view(count, opt['kNumberOfRotations'], opt['kNumberOfScales'], opt['kSide'] * opt['kSide'] * opt['kSide'])
        data_labels = torch.clone(loaded_data[:, -1]).squeeze() - 1
        data_data = torch.gt(data_data, 0).type(torch.FloatTensor)
        data_data = data_data.view(count, opt['kNumberOfRotations'], opt['kNumberOfScales'], 1, opt['kSide'], opt['kSide'], opt['kSide'])
        
        self.sampleLen = count
        self.datasets = data_data
        self.labels = data_labels
        
        print('--------------------------------')
        print('inputs', data_data.size())
        print('targets', data_labels.size())
        print('min target', torch.min(data_labels))
        print('max target', torch.max(data_labels))
        print('--------------------------------')

    def __getitem__(self, idx):
        return {
            "data": self.datasets[idx],
            "label": self.labels[idx]
        }
    
    @staticmethod
    def move_batch_to_device(batch, device):
        batch["data"] = batch["data"].to(device)
        #if batch["label"].dim() == 1:
        #    batch["label"] = batch["label"].unsqueeze(-1)
        batch["label"] = batch["label"].to(device)

    def __len__(self):
        return self.sampleLen

# Try loading a sample data point
temp = PointDataset('./data/benchmark/sg28_station4_intensity_rgb_train.txt_aggregated.txt', 1000, opt).__getitem__(0)
temp.keys()

torch.Size([1000, 20481])
--------------------------------
inputs torch.Size([1000, 1, 5, 1, 16, 16, 16])
targets torch.Size([1000])
min target tensor(0)
max target tensor(7)
--------------------------------


dict_keys(['data', 'label'])

In [7]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [8]:
class Reshape(torch.nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)
    
class Parallel(torch.nn.Module):
    def __init__(self, input_dimension, output_dimension, *args):
        super(Parallel, self).__init__()
        self.input_dimension = input_dimension
        self.output_dimension = output_dimension
        self.module_list = torch.nn.ModuleList(list(args))

    def forward(self, x):
        module_count = x.size(self.input_dimension)
        
        assert module_count == len(self.module_list), f"{len(self.module_list)} modules for {module_count} input parts"
        
        outputs = []
        totalOutputSize = None
        
        for i, l in enumerate(self.module_list):
            currentInput = torch.select(x, self.input_dimension, i)
            #print(currentInput.type())
            currentOutput = l(currentInput)
            #print(f'{i}: {currentInput.shape} -> {currentOutput.shape}')
            
            outputs.append(currentOutput)
            outputSize = currentOutput.size(self.output_dimension)
            
            if i == 0:
                totalOutputSize = list(currentOutput.size())
            else:
                totalOutputSize[self.output_dimension] = totalOutputSize[self.output_dimension] + outputSize
        
        output = torch.zeros(totalOutputSize).to(x.device)
        #print(f'Output size: {totalOutputSize}')
        
        offset = 0
        for i in range(module_count):
            currentOutput = outputs[i]
            outputSize = currentOutput.size(self.output_dimension)
            # TODO: verify if this works
            output.narrow(self.output_dimension, offset, outputSize).copy_(currentOutput)
            offset = offset + outputSize
            
        return output

class PointCloudModel(torch.nn.Module):
    def __init__(self, input_size, n_outputs, number_of_filters, number_of_scales, number_of_rotations):
        super(PointCloudModel, self).__init__()
        
        self.opt = opt
        self.input_size = input_size
        self.n_outputs = n_outputs
        self.number_of_filters = number_of_filters
        self.number_of_scales = number_of_scales
        self.number_of_rotations = number_of_rotations
        
        self.fcn_multiplier = 128
        
        self.full_model = torch.nn.Sequential(
            self.define_rotation_model(self.input_size, self.number_of_filters,
                                      self.number_of_scales, self.number_of_rotations),
            torch.nn.MaxPool2d((self.number_of_rotations, 1)),
            Reshape(-1, 4 * self.number_of_filters * self.number_of_scales * ((self.input_size // 8) ** 3)),
            torch.nn.Linear(
                4 * self.number_of_filters * self.number_of_scales * ((self.input_size // 8) ** 3),
                self.fcn_multiplier * self.number_of_filters
            ),
            torch.nn.ReLU(),
            torch.nn.Dropout(),
            torch.nn.Linear(self.fcn_multiplier * self.number_of_filters, self.n_outputs),
            torch.nn.LogSoftmax(dim=None)
        )
        
    def define_convolutional_model(self, input_size, number_of_filters):
        return torch.nn.Sequential(
            torch.nn.Conv3d(1, number_of_filters, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool3d(kernel_size=2, stride=2),
            torch.nn.Conv3d(number_of_filters, 2*number_of_filters, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool3d(kernel_size=2, stride=2),
            torch.nn.Conv3d(2*number_of_filters, 4*number_of_filters, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool3d(kernel_size=2, stride=2),
            Reshape(-1, 4 * number_of_filters * (int(input_size / 8) ** 3))
        )
    
    def define_scale_model(self, input_size, number_of_filters, number_of_scales):
        result = []
        for i in range(number_of_scales):
            result.append(self.define_convolutional_model(input_size, number_of_filters))
        
        return Parallel(1, 1, *tuple(result))
    
    def define_rotation_model(self, input_size, number_of_filters, number_of_scales, number_of_rotations):
        result = []
        #current_module = torch.nn.Sequential(
        #    self.define_scale_model(input_size, number_of_filters, number_of_scales),
        #    Reshape(-1, 1, 4 * number_of_filters * number_of_scales * ((input_size // 8) ** 3))
        #)
        for i in range(number_of_rotations):
            result.append(torch.nn.Sequential(
            self.define_scale_model(input_size, number_of_filters, number_of_scales),
            Reshape(-1, 1, 4 * number_of_filters * number_of_scales * ((input_size // 8) ** 3))
        ))
        return Parallel(1, 1, *tuple(result))

    def forward(self, x):
        b = x.size(0)
        x = self.full_model(x)
        return x

    def configure_optimizers(self):
        return torch.optim.Adadelta(self.parameters(), lr=0.1)
    
    def configure_loss(self):
        return torch.nn.NLLLoss()

# Create a test model
temp_device = torch.device('cpu')
temp_model = PointCloudModel(opt['kSide'], opt['n_outputs'], opt['number_of_filters'], opt['kNumberOfScales'], opt['kNumberOfRotations']).to(temp_device)
# Display parameter count
count_parameters(temp_model)
# Empty input matching the dataset shape
temp = torch.zeros(2, opt['kNumberOfRotations'], opt['kNumberOfScales'], 1, opt['kSide'], opt['kSide'], opt['kSide']).to(temp_device)
print(f'Input shape: {temp.shape}')
print(f'Output shape: {temp_model(temp).shape}')

+-----------------------------------------------------+------------+
|                       Modules                       | Parameters |
+-----------------------------------------------------+------------+
| full_model.0.module_list.0.0.module_list.0.0.weight |    432     |
|  full_model.0.module_list.0.0.module_list.0.0.bias  |     16     |
| full_model.0.module_list.0.0.module_list.0.3.weight |   13824    |
|  full_model.0.module_list.0.0.module_list.0.3.bias  |     32     |
| full_model.0.module_list.0.0.module_list.0.6.weight |   55296    |
|  full_model.0.module_list.0.0.module_list.0.6.bias  |     64     |
| full_model.0.module_list.0.0.module_list.1.0.weight |    432     |
|  full_model.0.module_list.0.0.module_list.1.0.bias  |     16     |
| full_model.0.module_list.0.0.module_list.1.3.weight |   13824    |
|  full_model.0.module_list.0.0.module_list.1.3.bias  |     32     |
| full_model.0.module_list.0.0.module_list.1.6.weight |   55296    |
|  full_model.0.module_list.0.0.mo

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [9]:
def train(model, trainloader, valloader, device, config):
    loss_criterion = model.configure_loss().to(device)
    optimizer = model.configure_optimizers()
    
    model.train()
    best_accuracy = 0.
    train_loss_running = 0.

    for epoch in range(config['max_epochs']):
        for i, batch in enumerate(trainloader):
            PointDataset.move_batch_to_device(batch, device)

            optimizer.zero_grad()
            prediction = model(batch['data'])
            loss_total = loss_criterion(prediction, batch['label'])
            loss_total.backward()
            optimizer.step()

            # loss logging
            train_loss_running += loss_total.item()
            iteration = epoch * len(trainloader) + i

            if iteration % config['print_every_n'] == (config['print_every_n'] - 1):
                print(f'[{epoch:03d}/{i:05d}] train_loss: {train_loss_running / config["print_every_n"]:.3f}')
                train_loss_running = 0.

            # validation evaluation and logging
            if iteration % config['validate_every_n'] == (config['validate_every_n'] - 1):

                # set model to eval, important if your network has e.g. dropout or batchnorm layers
                model.eval()

                loss_total_val = 0
                total, correct = 0, 0
                # forward pass and evaluation for entire validation set
                for batch_val in valloader:
                    PointDataset.move_batch_to_device(batch_val, device)

                    with torch.no_grad():
                        # TODO: Get prediction scores
                        prediction = model(batch_val['data'])

                    # Get predicted labels from scores
                    predicted_label = torch.max(prediction, dim=1)[1]

                    # keep track of total / correct / loss_total_val
                    total += predicted_label.shape[0]
                    correct += (predicted_label == batch_val['label']).sum().item()

                    loss_total_val += loss_criterion(prediction, batch_val['label']).item()

                accuracy = 100 * correct / total

                print(f'[{epoch:03d}/{i:05d}] val_loss: {loss_total_val / len(valloader):.3f}, val_accuracy: {accuracy:.3f}%')

                if accuracy > best_accuracy:
                    torch.save(model.state_dict(), f'runs/{config["experiment_name"]}/model_best.ckpt')
                    best_accuracy = accuracy

                # set model back to train
                model.train()


def main(config):
    # Declare device
    device = torch.device('cpu')
    if torch.cuda.is_available() and config['device'].startswith('cuda'):
        device = torch.device(config['device'])
        print('Using device:', config['device'])
    else:
        print('Using CPU')

    # Create Dataloaders
    train_dataset = PointDataset('./data/benchmark/sg28_station4_intensity_rgb_train.txt_aggregated.txt', 1000, config)
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    val_dataset = PointDataset('./data/benchmark/bildstein_station1_xyz_intensity_rgb_train.txt_aggregated.txt', 1000, config)
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # Instantiate model
    model = PointCloudModel(config['kSide'], config['n_outputs'], config['number_of_filters'], config['kNumberOfScales'], config['kNumberOfRotations'])

    # Load model if resuming from checkpoint
    if config['resume_ckpt'] is not None:
        model.load_state_dict(torch.load(config['resume_ckpt'], map_location='cpu'))

    # Move model to specified device
    model.to(device)

    # Create folder for saving checkpoints
    Path(f'runs/{config["experiment_name"]}').mkdir(exist_ok=True, parents=True)
    
    return model, train_dataloader, val_dataloader, device

In [10]:
opt = {
    'experiment_name': 'overfit',
    'max_epochs': 5000,
    'device': 'cuda:0',
    'resume_ckpt': None,
    'print_every_n': 100,
    'validate_every_n': 100,
    'kSide': cpp_options['kWindowSize'],
    'batch_size': cpp_options['kBatchSize'],
    'n_outputs': cpp_options['kNumberOfClasses'],
    'kNumberOfScales': cpp_options['kDefaultNumberOfScales'],
    'kNumberOfRotations': cpp_options['kDefaultNumberOfRotations'],
    'kSmallPrintingInterval': 2,
    'number_of_filters': 16,
    'kLargePrintingInterval': 100,
    'kWarmStart': False,
    'kModelDumpName': '../dump/model_dump',
    'kOptimStateDumpName': '../dump/optim_state_dump',
    'kStreamingPath': '../data/benchmark/sg28_station4_intensity_rgb_train.txt'
}

In [11]:
(model, train_dataloader, val_dataloader, train_device) = main(opt)

Using device: cuda:0
torch.Size([1000, 20481])
--------------------------------
inputs torch.Size([1000, 1, 5, 1, 16, 16, 16])
targets torch.Size([1000])
min target tensor(0)
max target tensor(7)
--------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([1000, 20481])
--------------------------------
inputs torch.Size([1000, 1, 5, 1, 16, 16, 16])
targets torch.Size([1000])
min target tensor(0)
max target tensor(7)
--------------------------------


In [12]:
train(model, train_dataloader, val_dataloader, train_device, opt)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[009/00009] train_loss: 1.501
[009/00009] val_loss: 2.176, val_accuracy: 38.400%
[019/00009] train_loss: 0.492
[019/00009] val_loss: 2.129, val_accuracy: 53.600%
[029/00009] train_loss: 0.228
[029/00009] val_loss: 2.185, val_accuracy: 57.100%
[039/00009] train_loss: 0.143
[039/00009] val_loss: 2.527, val_accuracy: 57.800%
[049/00009] train_loss: 0.094
[049/00009] val_loss: 2.768, val_accuracy: 57.400%
[059/00009] train_loss: 0.064
[059/00009] val_loss: 3.137, val_accuracy: 54.200%
[069/00009] train_loss: 0.044
[069/00009] val_loss: 2.964, val_accuracy: 59.100%
[079/00009] train_loss: 0.033
[079/00009] val_loss: 3.330, val_accuracy: 57.700%
[089/00009] train_loss: 0.023
[089/00009] val_loss: 3.235, val_accuracy: 59.800%
[099/00009] train_loss: 0.016
[099/00009] val_loss: 3.370, val_accuracy: 59.900%
[109/00009] train_loss: 0.014
[109/00009] val_loss: 3.449, val_accuracy: 56.400%
[119/00009] train_loss: 0.011
[119/00009] val_loss: 3.741, val_accuracy: 58.500%
[129/00009] train_loss: 0.00

KeyboardInterrupt: ignored

# Predicting

In the original implementation, they were using loader_ffi library which connects to libPointUtil.so, if we'd redesign, we'd still need point cloud / voxel grid libraries, therefore we've decided to keep this as is, so it will be the most compatible library for this model, usable in Python with the help of CFFI interface.

In [13]:
from cffi import FFI
import numpy as np


class FFIHandler:
    def __init__(self, lib_path, opt, testing_path=None, streaming_path=None, loader_seed=None):
        self.ffi = FFI()

        self.ffi.cdef("""
        struct DataLoaderArray;
        typedef struct DataLoaderWrapper {
          struct DataLoaderArray *inner_data_loader_ptr_;
        } DataLoaderWrapper;
        void construct(DataLoaderWrapper *self, int seed, const char *input_path, int number_of_scales, int number_of_rotations);
        void destruct(DataLoaderWrapper *self);
        void set_up_infinite_streaming(DataLoaderWrapper *self);
        float *next_random_batch(DataLoaderWrapper *self);
        int get_window_size();
        void set_up_test_streaming(DataLoaderWrapper *self);
        float *next_testing_batch(DataLoaderWrapper *self);
        """
                 )

        self.loader_lib = self.ffi.dlopen(lib_path)
        self.loader = self.ffi.new("DataLoaderWrapper *")
        self.testing_path = testing_path
        self.streaming_path = streaming_path
        self.loader_seed = loader_seed
        self.opt = opt

    def set_up_testing_loader(self):
        self.loader_lib.construct(self.loader, 1, self.testing_path.encode('ascii'), self.opt['kNumberOfScales'], self.opt['kNumberOfRotations'])
        self.loader_lib.set_up_test_streaming(self.loader)
    
    def set_up_loader(self, seed):
        # we are not using this, but just for completeness, if anyone
        # wants to load a pure file without aggregated input for training
        self.loader_lib.construct(self.loader, self.loader_seed, self.streaming_path.encode('ascii'), self.opt['kNumberOfScales'], self.opt['kNumberOfRotations'])
        self.loader_lib.set_up_infinite_streaming(self.loader)

    def transform(self, pointer):
        batch_size = pointer[0]
        if batch_size != self.opt['batch_size']:
            batch = torch.FloatTensor((self.opt['kNumberOfRotations'] * self.opt['kNumberOfScales'] * (self.opt['kSide'] ** 3) + 1) * batch_size)
        else:
            batch = torch.FloatTensor((self.opt['kNumberOfRotations'] * self.opt['kNumberOfScales'] * (self.opt['kSide'] ** 3) + 1) * self.opt['batch_size'])
      
        array = np.zeros_like(batch)
        pointex = self.ffi.from_buffer("float[]", array)

        self.ffi.memmove(pointex, pointer + 1, batch.nelement() * batch.element_size())

        loaded_tensor = torch.from_numpy(array)
        count = batch_size if batch_size != self.opt['batch_size'] else self.opt['batch_size']
        
        loaded_data = loaded_tensor.view(count, -1)
        data_data = torch.clone(loaded_data[:, :-1]).view(count, opt['kNumberOfRotations'], opt['kNumberOfScales'], opt['kSide'] * opt['kSide'] * opt['kSide'])
        data_labels = torch.clone(loaded_data[:, -1]).squeeze() - 1
        data_data = torch.gt(data_data, 0).type(torch.FloatTensor)
        data_data = data_data.view(count, opt['kNumberOfRotations'], opt['kNumberOfScales'], 1, opt['kSide'], opt['kSide'], opt['kSide'])

        # if batch_size != self.opt['batch_size']:
        #     batch = batch.view(batch_size, opt['kNumberOfRotations'], opt['kNumberOfScales'], 1, opt['kSide'], opt['kSide'], opt['kSide'])
        #     label = torch.zeros(batch_size)
        # else:
        #     batch = batch.view(self.opt['batch_size'], opt['kNumberOfRotations'], opt['kNumberOfScales'], 1, opt['kSide'], opt['kSide'], opt['kSide'])
        #     label = torch.zeros(self.opt['batch_size'])

        return {"data":data_data, "label":data_labels}

    def get_next_testing_batch(self):
        pointer = self.loader_lib.next_testing_batch(self.loader)
        if pointer[0] > 0:
            return self.transform(pointer)
        else:
            return None
    
    def get_next_random_batch(self):
        # we are not using this, but just for completeness, if anyone
        # wants to load a pure file without aggregated input for training
        pointer = self.loader_lib.next_random_batch(self.loader)
        return self.transform(pointer)

    def get_window_size(self):
        # we are not using this, but just for completeness
        return self.loader_lib.get_window_size()


ffi_handler = FFIHandler('/content/semantic3dnet/build/lib/point_cloud_util/libpointCloudUtil.so', opt, testing_path='/content/semantic3dnet/data/benchmark/MarketplaceFeldkirch_Station4_rgb_intensity-reduced_test.txt')
ffi_handler.set_up_testing_loader()

In [ ]:
def predict(model, valloader, device, config):
    model.eval()

    loss_total_val = 0
    total, correct = 0, 0
    
    f = open("results.txt", 'w')
    while True:
        batch = ffi_handler.get_next_testing_batch()
        if batch == None:
           break
        
        PointDataset.move_batch_to_device(batch, device)

        with torch.no_grad():
            prediction = model(batch['data'])

        predicted_label = torch.max(prediction, dim=1)[1]
        # print(predicted_label) # you can print the labels if you want.

        # writing into the results as Semantic3Dnet label format.
        predicted_label = predicted_label.cpu().numpy()
        for i in range(len(predicted_label)):
            f.write(str(predicted_label[i]) + "\n")


    
    f.close()


predict(model, None, train_device, opt)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
